In [2]:
from tabula import read_pdf
import os
import pandas as pd

#### Parsa i PDF.
Salva un csv per pdf con le colonne: speed, time.
Il nome del file è: targa_voucher_idcrash_giorno-mese-anno-ore-minuti-secondi.csv
Per Viasat è necessario ricampionare al secondo le velocità interpolando valori vicini.

Legge i pdf dalla cartella salvata nella variabile directory,
e li salva nella sottocartella "CSV" (da creare manualmente).

In [28]:
directory = './PDF_NON_ANOM'
count_file = 0
for root,_,files in os.walk(directory):
    for file in files:
        if not file.endswith('pdf'):
            continue
        count_file += 1
        try:
            dff = pd.DataFrame()
            if len(file.split('_'))==5: # vodafone
                print(count_file, 'vodafone')
                df = read_pdf(os.path.join(root,file), pages=8).iloc[1:]
                dff['speed'] = df['Velocità']
                dff['time'] = df['Tempo'].apply(lambda r: r.split()[1])
                # pag1
                df = read_pdf(os.path.join(root,file), pages=1)
                #print(df)
                targa = df.iloc[2][0].strip('Targa ')
                voucher = df.iloc[7][4]
                # pag 2
                df = read_pdf(os.path.join(root,file), pages=2)
                idcrash = df.iloc[0][1]
                data = df.iloc[1][1].split()[1].replace('/','-')
                ora = df.iloc[1][1].split()[2].replace(':','-')
                file_name = '_'.join([targa,str(voucher),str(idcrash),data,ora])
                dff.to_csv('CSV\\'+file_name+'.csv', sep=';', index = False)
            else:
                df = read_pdf(os.path.join(root,file), pages=4)
                if 'DOSSIER SCATOLA NERA' in list(df.columns): # viasat
                    print(count_file, 'viasat')
                    df = df.iloc[5:]
                    dff['speed'] = df['DOSSIER SCATOLA NERA'].apply(lambda r: float(r.split()[1].replace(',','.')))
                    dff['time'] = df['DOSSIER SCATOLA NERA'].apply(lambda r: r.split()[0].replace('.',':'))
                    dff['time'] = pd.to_datetime(dff['time'], format='%H:%M:%S')
                    dff = dff.set_index('time').resample('1S').interpolate(method='linear').reset_index()
                    dff['time'] = dff['time'].apply(lambda r: str(r).split()[1])
                    dff = dff.iloc[14:55]
                    # pag1
                    df = read_pdf(os.path.join(root,file), pages=1)
                    data = df.iloc[20][0].strip('Data ').replace('/','-')
                    ora = df.iloc[20][1][5:12].replace('.','-')
                    targa = df.iloc[7][0].strip('Targa ')
                    voucher = df.iloc[5][2]
                    # pag2
                    df = read_pdf(os.path.join(root,file), pages=2)
                    idcrash = df.iloc[0][3]
                    file_name = '_'.join([targa,str(voucher),str(idcrash),data,ora])
                    dff.to_csv('CSV\\'+file_name+'.csv', sep=';', index = False)
                else:  # infomobiliy
                    print(count_file, 'infomobility')
                    #dff['speed'] = df['Velocità Km/h']
                    #dff['time'] = df['Data'].apply(lambda r: r.split()[1])
                    #dff.to_csv('CSV\\'+file.rstrip('.pdf')+'.csv', sep=';')
        except Exception as e:
            print('errore {} su file {}'.format(e,file))

1 viasat
2 vodafone
